In [ ]:
import os
import tensorflow as tf
import numpy as np
from google.colab import drive
#set seed
SEED=1234
tf.random.set_seed(SEED)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!unzip '/content/drive/My Drive/AN2DL/save_model/Inception_final.zip' -d '/content/drive/My Drive/AN2DL/save_model'
#!unzip '/content/drive/My Drive/AN2DL/save_model/BestResnetFabio.zip' -d '/content/drive/My Drive/AN2DL/save_model'

In [ ]:
#loading our three best models
xception_model = tf.keras.models.load_model('/content/drive/My Drive/AN2DL/save_model/Xception_final')
Inception_model = tf.keras.models.load_model('/content/drive/My Drive/AN2DL/save_model/Inception_final')
Resnet_model = tf.keras.models.load_model('/content/drive/My Drive/AN2DL/save_model/Resnet_final')

In [ ]:
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet_v2 import preprocess_input as preprocess_input_resnet
from keras.applications.xception import preprocess_input as preprocess_input_xception
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input as preprocess_input_inception

import statistics
final_path = os.path.join('/content/drive/My Drive/AN2DL/MaskDataset/', 'test')
image_filenames = next(os.walk(final_path))[2]  
img_h=512
img_w=512
results = {} 
for image_name in image_filenames: 
   img = Image.open(final_path+'/'+image_name).convert('RGB')
   img=img.resize((img_h,img_w))
   img_array = np.array(img)
   #compute prediction with Resnet model
   img_array1= preprocess_input_resnet(img_array) 
   img_array1 = np.expand_dims(img_array1, 0)
   prediction1=Resnet_model.predict(img_array1)
   #compute prediction with Inception model
   img_array2= preprocess_input_inception(img_array) 
   img_array2 = np.expand_dims(img_array2, 0)
   prediction2=Inception_model.predict(img_array2)
   #compute prediction with Xception model
   img_array3= preprocess_input_xception(img_array) 
   img_array3 = np.expand_dims(img_array3, 0)
   prediction3=xception_model.predict(img_array3)
   #compute the average between models
   results[image_name]=np.argmax(1/3*prediction1+1/3*prediction2+1/3*prediction3)

In [ ]:
#creating the csv
import os
from datetime import datetime
def create_csv(results, results_dir='./'):

    csv_fname = 'ensembling_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')
create_csv(results,'/content/drive/My Drive/AN2DL')